In [1]:
import re
import qiskit
import json
import time

import numpy as np
from typing import List
from qiskit import transpile
import time

from fast_generator import fc_tree_commute_recur_lookahead_fast

from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
from circuit_generator import construct_qcc_circuit

import numpy as np
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

In [2]:
#Compare json files in a specific folder
import os, json
def run_experiment_folder(folder_path = None, filename = None, save_output = False, threshold = 1):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)

            # Function to check if a string contains only 'I' characters
            def is_all_identity(pauli):
                return all(char == 'I' for char in pauli)

            # Filter the list to remove all identity Paulis
            test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for our method
            start_time = time.time()
            opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            opt_qc_f = transpile(opt_qc_f, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=FakeSherbrooke().coupling_map)
            end_time = time.time()
            our_time = end_time - start_time

            start_time = time.time()
            opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=FakeSherbrooke().coupling_map)
            end_time = time.time()
            combined_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": len(test_paulis),
                "times": {
                    "our_time": our_time,
                    "combined_time": combined_time
                },
                "gate_counts": {
                    "our_method": opt_qc_f.count_ops().get('cx', 0),
                    "combined_method": opt_qiskit.count_ops().get('cx', 0)
                },
                "circuit_entangling_depth": {
                    "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
                    "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1)
                },
                "test_paulis_file": f'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_' + filename, 'w') as paulis_file:
                    json.dump([test_paulis, results], paulis_file, indent=4)
    

    

In [3]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, threshold = 1, filename = "Paulis"):

    results = []

    def is_all_identity(pauli):
        return all(char == 'I' for char in pauli)
    paulis = test_paulis
    # Filter the list to remove all identity Paulis
    test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]

    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    opt_qc_f = transpile(opt_qc_f, optimization_level=0, basis_gates=["cx", "sx", "x", "rz"], coupling_map=FakeSherbrooke().coupling_map)
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"], coupling_map=FakeSherbrooke().coupling_map)
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0)
        },
        "circuit_entangling_depth": {
            "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
            "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1)
        },
        "test_paulis_file": f'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_' + filename, 'w') as paulis_file:
            json.dump([test_paulis, results], paulis_file, indent=4)
    return sorted_entanglers_f
    

    

In [4]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,]# 12]
orbitals_list = [4, 6, 8, 12, 16, 20,]# 24]
#First evaluate the UCCSD ansatz:
for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = True, threshold = 1, filename=f"Paulis{len(test_paulis)}.json")

{'num_paulis': 24, 'times': {'our_time': 0.6053826808929443, 'combined_time': 0.5644912719726562}, 'gate_counts': {'our_method': 42, 'combined_method': 28}, 'circuit_entangling_depth': {'our_method': 35, 'combined_method': 22}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_Paulis24.json'}
{'num_paulis': 80, 'times': {'our_time': 2.1133508682250977, 'combined_time': 1.6077044010162354}, 'gate_counts': {'our_method': 392, 'combined_method': 191}, 'circuit_entangling_depth': {'our_method': 281, 'combined_method': 164}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_Paulis80.json'}
{'num_paulis': 320, 'times': {'our_time': 6.516584873199463, 'combined_time': 5.4444520473480225}, 'gate_counts': {'our_method': 2007, 'combined_method': 1022}, 'circuit_entangling_depth': {'our_method': 1288, 'combined_method': 780}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_Paulis320.json'}
{'num_paul

In [5]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = True, threshold = 1)

benzene.json
{'num_paulis': 1254, 'times': {'our_time': 68.7208731174469, 'combined_time': 49.95949983596802}, 'gate_counts': {'our_method': 17623, 'combined_method': 9872}, 'circuit_entangling_depth': {'our_method': 8934, 'combined_method': 6444}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_benzene.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 7.181808710098267, 'combined_time': 5.11433482170105}, 'gate_counts': {'our_method': 1396, 'combined_method': 773}, 'circuit_entangling_depth': {'our_method': 865, 'combined_method': 604}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_H2O.json'}
LiH.json
{'num_paulis': 61, 'times': {'our_time': 2.5860893726348877, 'combined_time': 1.37831711769104}, 'gate_counts': {'our_method': 277, 'combined_method': 155}, 'circuit_entangling_depth': {'our_method': 206, 'combined_method': 146}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_

In [9]:
#Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, threshold = 1)

max_cut_benchmark_connected_n10_e12_l1.json
{'num_paulis': 22, 'times': {'our_time': 1.0073094367980957, 'combined_time': 1.1638894081115723}, 'gate_counts': {'our_method': 266, 'combined_method': 72}, 'circuit_entangling_depth': {'our_method': 127, 'combined_method': 54}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_max_cut_benchmark_connected_n10_e12_l1.json'}
max_cut_benchmark_connected_n10_e12_l3.json
{'num_paulis': 66, 'times': {'our_time': 2.3217952251434326, 'combined_time': 1.8580341339111328}, 'gate_counts': {'our_method': 656, 'combined_method': 221}, 'circuit_entangling_depth': {'our_method': 324, 'combined_method': 138}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_max_cut_benchmark_connected_n10_e12_l3.json'}
max_cut_benchmark_connected_n15_e63_l1.json
{'num_paulis': 78, 'times': {'our_time': 5.4983673095703125, 'combined_time': 6.1385979652404785}, 'gate_counts': {'our_method': 1025, 'combined_metho

In [7]:
#since in the maxcut problems there are many commuting terms and it's slow to search through all of them,
#we can set the threshold to 2 to early stop the search once found a simplified pauli of weight 2
#you can see the run time difference, for the second benchmark runtime reduces from 30second to 7 seconds.

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, threshold = 2)

max_cut_benchmark_connected_n10_e12_l1.json
{'num_paulis': 22, 'times': {'our_time': 1.015287160873413, 'combined_time': 0.9434778690338135}, 'gate_counts': {'our_method': 296, 'combined_method': 67}, 'circuit_entangling_depth': {'our_method': 142, 'combined_method': 47}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_max_cut_benchmark_connected_n10_e12_l1.json'}
max_cut_benchmark_connected_n10_e12_l3.json
{'num_paulis': 66, 'times': {'our_time': 2.1143500804901123, 'combined_time': 1.808168649673462}, 'gate_counts': {'our_method': 593, 'combined_method': 224}, 'circuit_entangling_depth': {'our_method': 291, 'combined_method': 151}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_max_cut_benchmark_connected_n10_e12_l3.json'}
max_cut_benchmark_connected_n15_e63_l1.json
{'num_paulis': 78, 'times': {'our_time': 4.31846022605896, 'combined_time': 4.5588178634643555}, 'gate_counts': {'our_method': 1179, 'combined_method': 

In [8]:
#Labs

run_experiment_folder(folder_path = "benchmarks/labs_paulis", save_output = True, threshold= 2)

labs_n10_layers1.json
{'num_paulis': 80, 'times': {'our_time': 4.474046945571899, 'combined_time': 3.499645948410034}, 'gate_counts': {'our_method': 841, 'combined_method': 377}, 'circuit_entangling_depth': {'our_method': 429, 'combined_method': 274}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_labs_n10_layers1.json'}
labs_n10_layers3.json
{'num_paulis': 240, 'times': {'our_time': 11.443421602249146, 'combined_time': 9.096692085266113}, 'gate_counts': {'our_method': 2315, 'combined_method': 1071}, 'circuit_entangling_depth': {'our_method': 1197, 'combined_method': 745}, 'test_paulis_file': 'benchmarks/results_hardware_topology_ibm_sherbrooke/test_new_labs_n10_layers3.json'}
labs_n11_layers1.json
{'num_paulis': 106, 'times': {'our_time': 6.215391635894775, 'combined_time': 5.384611368179321}, 'gate_counts': {'our_method': 1187, 'combined_method': 503}, 'circuit_entangling_depth': {'our_method': 589, 'combined_method': 354}, 'test_paulis_file': 'benc